In [6]:
import pandas as pd
from sqlalchemy import create_engine

server = '127.0.0.1'
database = 'Actividad_ETL'
username = 'sa'
password = '12345678'
conn_str = f"mssql+pymssql://{username}:{password}@{server}/{database}"
engine = create_engine(conn_str)
query = "SELECT * FROM dbo.tabla_etl_nueva1"
df = pd.read_sql(query, engine)

df.info()  # Tipos de datos, valores nulos

# Renombrar columnas: eliminar espacios y caracteres especiales
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()

# Variables numéricas reales
numerical_columns = ["snies", "ano", "puntaje_global_icfes", "a_global", "a_lectura_critica", 
                     "a_matematicas", "a_sociales_y_ciudadanas", 
                     "a_ciencias_naturales", "a_ingles", "edad"]

# Aplicar conversiones
df[numerical_columns] = df[numerical_columns].astype("float64")
df[["codigo_estudiante"]] = df[["codigo_estudiante"]].astype("object")
df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).astype("category")

df.fillna({col: "No Registra" for col in df.select_dtypes(include=['object']).columns}, inplace=True)
df.fillna({col: 0 for col in df.select_dtypes(include=['number']).columns}, inplace=True)

mediana_edad = df["edad"].median()
df.loc[df["edad"] == 0, "edad"] = mediana_edad

columnas_categoricas = df.select_dtypes(include=['category']).columns

coincidencias = (df["registro_snp"].astype(str) == df["snp"].astype(str)).sum()
diferencias = (df["registro_snp"].astype(str) != df["snp"].astype(str)).sum()

df["puntaje_global_icfes"] = pd.to_numeric(df["puntaje_global_icfes"], errors="coerce")
df["a_global"] = pd.to_numeric(df["a_global"], errors="coerce")
if (df["puntaje_global_icfes"] == df["a_global"]).all():
    df.drop(columns=["a_global"], inplace=True)

# Convertimos las columnas categóricas a string antes de reemplazar valores
df["estado_civil"] = df["estado_civil"].astype(str).replace(
    {"No registra": "No Registra", "Sin Registro": "No Registra", "Unión libre": "Unión Libre"}
).astype("category")

df["colegio_sector"] = df["colegio_sector"].astype(str).replace(
    {"NO REGISTRA": "No Registra", "SIN CLASIFICACION": "Sin Clasificación"}
).astype("category")

df["colegio_clasificacion"] = df["colegio_clasificacion"].astype(str).replace(
    {"NO REGISTRA": "No Registra", 
     "SIN CLASIFICACION": "Sin Clasificación",
     "SIN CLASIFICACIÓN": "Sin Clasificación"}  # Normaliza acentos
).astype("category")

df['codigo_estudiante'] = pd.to_numeric(df['codigo_estudiante'].astype(str), errors='coerce').astype('Int64')

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.upper()

# Convertir columnas categóricas a mayúsculas evitando duplicados
columnas_categoricas = df.select_dtypes(include=['category']).columns
for col in columnas_categoricas:
    # Asegurar que todas las categorías sean strings y eliminar NaN antes de aplicar str.upper()
    categorias_limpias = df[col].cat.categories.dropna().astype(str)
    
    # Convertir a mayúsculas y eliminar duplicados
    categorias_mayusculas = list(set(categorias_limpias.str.upper()))
    
    # Asignar las nuevas categorías
    df[col] = df[col].cat.set_categories(categorias_mayusculas)

# Carga de datos transformados a SQL
df.to_sql("TRANSFORMACION", engine, if_exists="replace", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11729 entries, 0 to 11728
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   codigo_estudiante              11729 non-null  float64
 1   Registro_SNP                   11729 non-null  object 
 2   Periodo_Presentacion1          11729 non-null  object 
 3   Periodo_Presentacion           11729 non-null  object 
 4   Puntaje_Global_ICFES           11729 non-null  float64
 5   A_Global                       11729 non-null  float64
 6   A_Lectura_Critica              11729 non-null  float64
 7   A_Matematicas                  11729 non-null  float64
 8   A_Sociales_y_Ciudadanas        11729 non-null  float64
 9   A_Ciencias_Naturales           11729 non-null  float64
 10  A_Ingles                       11729 non-null  float64
 11  A_Nivel_Ingles                 11729 non-null  object 
 12  Periodo_Academico              11727 non-null 

16